In [11]:
# Restart kernel and reload modules
import importlib
import sys
if 'fugu.bricks.lca_bricks' in sys.modules:
    importlib.reload(sys.modules['fugu.bricks.lca_bricks'])
if 'fugu.bricks' in sys.modules:
    importlib.reload(sys.modules['fugu.bricks'])


In [12]:
# Force reload the module
import sys
import importlib

# Remove modules from cache
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('fugu')]
for module in modules_to_remove:
    del sys.modules[module]

# Re-import everything fresh
from fugu.bricks import LCABrick
from fugu.backends import slca_Backend  
import numpy as np
from fugu import Scaffold

Phi_demo = np.array([
    [0.3313, 0.8148, 0.4364],
    [0.8835, 0.3621, 0.2182],
    [0.3313, 0.4527, 0.8729],
], dtype=float)
s_demo = np.array([0.5, 1.0, 1.5], dtype=float)
lam_demo = 0.1

scaffold = Scaffold()

scaffold.add_brick(
    LCABrick(Phi=Phi_demo),
    output=True
)

scaffold.lay_bricks()

In [13]:
graph = scaffold.graph
print(f"Graph has {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges")

lca_neurons = [n for n in graph.nodes() if 'neuron_' in n]
print(f"Found {len(lca_neurons)} LCA neurons:")
for i, neuron in enumerate(lca_neurons): 
    print(f"  {neuron}: {graph.nodes[neuron]}")

print(f"\nExample edges (inhibitory connections):")
for i, (src, dst, data) in enumerate(graph.edges(data=True)):
    print(f"  {src} -> {dst}: weight={data.get('weight', 'N/A')}, delay={data.get('delay', 'N/A')}")

Graph has 4 nodes and 6 edges
Found 3 LCA neurons:
  Brick-0:neuron_0: {'index': 0, 'threshold': 1.0, 'reset_voltage': 0.0, 'leakage_constant': 1.0, 'potential': 0.0, 'bias': 0.0, 'p': 1.0, 'dt': 0.001, 'tau_syn': 1.0, 'lam': 0.1, 'neuron_type': 'CompetitiveNeuron', 'brick': 'Brick-0', 'neuron_number': 1}
  Brick-0:neuron_1: {'index': 1, 'threshold': 1.0, 'reset_voltage': 0.0, 'leakage_constant': 1.0, 'potential': 0.0, 'bias': 0.0, 'p': 1.0, 'dt': 0.001, 'tau_syn': 1.0, 'lam': 0.1, 'neuron_type': 'CompetitiveNeuron', 'brick': 'Brick-0', 'neuron_number': 2}
  Brick-0:neuron_2: {'index': 2, 'threshold': 1.0, 'reset_voltage': 0.0, 'leakage_constant': 1.0, 'potential': 0.0, 'bias': 0.0, 'p': 1.0, 'dt': 0.001, 'tau_syn': 1.0, 'lam': 0.1, 'neuron_type': 'CompetitiveNeuron', 'brick': 'Brick-0', 'neuron_number': 3}

Example edges (inhibitory connections):
  Brick-0:neuron_0 -> Brick-0:neuron_1: weight=0.7398216888397283, delay=1
  Brick-0:neuron_0 -> Brick-0:neuron_2: weight=0.6265187628748929

In [14]:
# Reload the backend with fixes
import importlib
importlib.reload(sys.modules['fugu.backends.slca_backend'])
from fugu.backends import slca_Backend

# Re-create and compile the backend with fixes
backend = slca_Backend()
backend.compile(
    scaffold=scaffold,
    compile_args={
        'y': s_demo,
        'Phi': Phi_demo,
        'lam': lam_demo,
        'dt': 1e-3,
        'tau_syn': 1.0,
        'T_steps': 100000,  # Reduce for testing
        't0_steps': 1000,
    }
)
print(f"Neuron biases set to: {backend.b}")
print(f"S-LCA parameters: dt={backend.dt}, tau={backend.tau}, lam={backend.lam}")
print(f"Network dimensions: {backend.N} neurons")

# Verify neuron parameters
print(f"\nNeuron parameters verification:")
for name, neuron in backend.nn.nrns.items():
    if "neuron_" in name and "complete" not in name:
        print(f"  {name}: bias={getattr(neuron, '_b', 'not set'):.4f}, "
              f"lam={getattr(neuron, 'lam', 'not set'):.3f}, "
              f"dt={getattr(neuron, 'dt', 'not set'):.3f}")

Neuron biases set to: [1.54602917 1.44858423 1.74574074]
S-LCA parameters: dt=0.001, tau=1.0, lam=0.1
Network dimensions: 3 neurons

Neuron parameters verification:
  Brick-0:neuron_0: bias=1.5460, lam=0.100, dt=0.001
  Brick-0:neuron_1: bias=1.4486, lam=0.100, dt=0.001
  Brick-0:neuron_2: bias=1.7457, lam=0.100, dt=0.001


In [15]:
results = backend.run()

In [16]:
# Reload backend with the array division fix
import importlib
importlib.reload(sys.modules['fugu.backends.slca_backend'])
from fugu.backends import slca_Backend


print("🔄 Backend reloaded and recompiled with array division fix!")

# Now run the S-LCA algorithm
print("🚀 Running S-LCA algorithm...")
results = backend.run()

print("✅ S-LCA completed successfully!")
print(f"\n📊 Results:")
print(f"  Activation coefficients (a_tail): {results['a_tail']}")
print(f"  Spike counts: {results['counts']}")
print(f"  Spike rates (Hz): {results['a_rate']}")
print(f"  Reconstruction x̂: {results['x_hat']}")
print(f"  Original signal y: {s_demo}")
print(f"  Reconstruction error: {np.linalg.norm(results['x_hat'] - s_demo):.6f}")

print(f"\n🔧 S-LCA Parameters:")
print(f"  Feedforward biases b: {results['b']}")
print(f"  Inhibition matrix shape: {results['W'].shape}")
print(f"  λ threshold: {backend.lam}")
print(f"  Simulation time: {backend.T_steps * backend.dt:.3f} seconds")
print(f"  Time steps: {backend.T_steps}")

# Verify the S-LCA worked correctly
sparsity = np.sum(results['a_tail'] > 0) / len(results['a_tail'])
print(f"\n📈 Analysis:")
print(f"  Sparsity (fraction of active coefficients): {sparsity:.2f}")
print(f"  Max activation coefficient: {np.max(results['a_tail']):.4f}")
print(f"  Mean activation coefficient: {np.mean(results['a_tail']):.4f}")

🔄 Backend reloaded and recompiled with array division fix!
🚀 Running S-LCA algorithm...
✅ S-LCA completed successfully!

📊 Results:
  Activation coefficients (a_tail): [0.67978066 0.         1.22275485]
  Spike counts: [ 67   0 122]
  Spike rates (Hz): [0.67 0.   1.22]
  Reconstruction x̂: [0.7588084  0.86736239 1.29253806]
  Original signal y: [0.5 1.  1.5]
  Reconstruction error: 0.357232

🔧 S-LCA Parameters:
  Feedforward biases b: [1.54602917 1.44858423 1.74574074]
  Inhibition matrix shape: (3, 3)
  λ threshold: 0.1
  Simulation time: 100.000 seconds
  Time steps: 100000

📈 Analysis:
  Sparsity (fraction of active coefficients): 0.67
  Max activation coefficient: 1.2228
  Mean activation coefficient: 0.6342
✅ S-LCA completed successfully!

📊 Results:
  Activation coefficients (a_tail): [0.67978066 0.         1.22275485]
  Spike counts: [ 67   0 122]
  Spike rates (Hz): [0.67 0.   1.22]
  Reconstruction x̂: [0.7588084  0.86736239 1.29253806]
  Original signal y: [0.5 1.  1.5]
  Rec